In [44]:
import pandas as pd
import numpy as np

In [45]:
df10 = pd.read_csv('./01_raw_data/EV_project/기후·환경/미세먼지10.csv', encoding='cp949')
df25 = pd.read_csv('./01_raw_data/EV_project/기후·환경/미세먼지2.5.csv', encoding='cp949')

In [46]:


# 2) '구분(1)' 컬럼명을 '지역'으로 변경
df25 = df25.rename(columns={"구분(1)": "지역"})
df10 = df10.rename(columns={"구분(1)": "지역"})

# 3) wide -> long 변환
df25_long = df25.melt(
    id_vars=["지역"],
    var_name="년월",
    value_name="미세먼지2.5 수치"
    
)
df10_long = df10.melt(
    id_vars=["지역"],
    var_name="년월",
    value_name="미세먼지10 수치"
)

# 4) '년월' 열을 연도(year)와 월(month)로 분리
df25_long[["년", "월"]] = (
    df25_long["년월"]
    .str.split(r"\.", expand=True)    # 'YYYY.MM' 또는 'YYYY.M' 형태
    .astype(int)
)
df10_long[["년", "월"]] = (
    df10_long["년월"]
    .str.split(r"\.", expand=True)
    .astype(int)
)

# 5) 원본 '년월' 컬럼 제거
df25_long = df25_long.drop(columns="년월")
df10_long = df10_long.drop(columns="년월")

# 6) 두 데이터프레임 병합 (inner join)
df = pd.merge(
    df25_long,
    df10_long,
    on=["지역", "년", "월"],
    how="inner"
)

df["미세먼지2.5 수치"] = (
    df["미세먼지2.5 수치"]
    .astype(str)                     # 숫자를 문자열로 변환
    .str.replace("*", "", regex=False)  # "*" 제거
    .replace("", pd.NA)             # 빈 문자열을 NaN으로
    .astype(float)                  # 다시 실수형으로 변환
)
df["미세먼지10 수치"] = (
    df["미세먼지10 수치"]
    .astype(str)                     # 숫자를 문자열로 변환
    .str.replace("*", "", regex=False)  # "*" 제거
    .replace("", pd.NA)             # 빈 문자열을 NaN으로
    .astype(float)                  # 다시 실수형으로 변환
)

# 7) 정렬 (원하는 순서대로)

df = df[["지역", "년", "월", "미세먼지2.5 수치", "미세먼지10 수치"]]
df = df.replace('총계', '전국')
df['지역'] = df['지역'].str.replace(
    r'(광역시|특별자치시|특별시|특별자치도|도)$', 
    '', 
    regex=True
)
abbr_map = {
    '전라북': '전북',
    '전라남': '전남',
    '경상북': '경북',
    '경상남': '경남',
    '충청북': '충북',
    '충청남': '충남'
}

# df 에 적용
df['지역'] = df['지역'].replace(abbr_map)

# 확인
print(df['지역'].unique())
# 8) 결과 확인
df


['전국' '서울' '부산' '대구' '인천' '광주' '대전' '울산' '세종' '경기' '강원' '충북' '충남' '전북'
 '전남' '경북' '경남' '제주']


,지역,년,월,미세먼지2.5 수치,미세먼지10 수치
0,전국,2018,11,32.0,56.0
1,서울,2018,11,28.0,52.0
2,부산,2018,11,26.0,47.0
3,대구,2018,11,31.0,57.0
4,인천,2018,11,29.0,55.0
...,...,...,...,...,...
1327,전북,2024,12,16.0,29.0
1328,전남,2024,12,12.0,22.0
1329,경북,2024,12,17.0,30.0
1330,경남,2024,12,13.0,21.0


In [47]:
# df25_long 에서 중복 키 개수
dups25 = df25_long.duplicated(subset=['지역','년','월'], keep=False).sum()
# df10_long 에서 중복 키 개수
dups10 = df10_long.duplicated(subset=['지역','년','월'], keep=False).sum()
print(f"df25_long 중복: {dups25}개, df10_long 중복: {dups10}개")

df25_long 중복: 0개, df10_long 중복: 0개


In [48]:
# df25_long 에서 각 키 조합별 개수 집계
dup_counts25 = (
    df25_long
    .groupby(['지역','년','월'])
    .size()
    .reset_index(name='cnt')
)

# cnt > 1 인 것만 필터
duplicates25 = dup_counts25[dup_counts25['cnt'] > 1]
print("df25_long 중복 키 조합과 개수:")
duplicates25


df25_long 중복 키 조합과 개수:


,지역,년,월,cnt


In [49]:
# 1) df25_long 에서 “미세먼지2.5 수치” 중복 키별 고유값 개수 확인
check25 = (
    df25_long
    .groupby(['지역','년','월'], as_index=False)['미세먼지2.5 수치']
    .agg(n_unique='nunique', unique_vals=lambda x: list(x.unique()))
)

# 2) n_unique > 1 인 경우만 남기기 → 값이 다른 키 조합
mismatch25 = check25[check25['n_unique'] > 1]
print("미세먼지2.5, 값이 섞여 있는 키 조합:")
print(mismatch25)

# 3) df10_long 도 동일하게
check10 = (
    df10_long
    .groupby(['지역','년','월'], as_index=False)['미세먼지10 수치']
    .agg(n_unique='nunique', unique_vals=lambda x: list(x.unique()))
)
mismatch10 = check10[check10['n_unique'] > 1]
print("\n미세먼지10, 값이 섞여 있는 키 조합:")
print(mismatch10)

# 4) 결과 해석
if mismatch25.empty:
    print("\n✅ df25_long의 중복 그룹 내 값들은 모두 동일합니다.")
else:
    print(f"\n⚠️ df25_long에서 {len(mismatch25)}개 키조합이 서로 다른 값들을 가지고 있습니다.")

if mismatch10.empty:
    print("✅ df10_long의 중복 그룹 내 값들도 모두 동일합니다.")
else:
    print(f"⚠️ df10_long에서 {len(mismatch10)}개 키조합이 서로 다른 값들을 가지고 있습니다.")


미세먼지2.5, 값이 섞여 있는 키 조합:
Empty DataFrame
Columns: [지역, 년, 월, n_unique, unique_vals]
Index: []

미세먼지10, 값이 섞여 있는 키 조합:
Empty DataFrame
Columns: [지역, 년, 월, n_unique, unique_vals]
Index: []

✅ df25_long의 중복 그룹 내 값들은 모두 동일합니다.
✅ df10_long의 중복 그룹 내 값들도 모두 동일합니다.


In [50]:
df.to_csv('./02_interim_data/dust_data.csv', index=False, encoding='utf-8-sig')